In [1]:
!pip install keras
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=6ccf46dc2ca46081f0671839f40297862a6feed3a1d2562650bbd07b25d3cbf6
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [2]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras import regularizers
from keras import layers, models
from tensorflow.keras import optimizers
from tensorflow.keras.applications import VGG16
from keras import Input
from keras.models import Model
from keras import initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint
import os
from glob import glob
from PIL import Image
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

data_dir = "/content/drive/My Drive/DeepLearningProject/model/data"
categories = []

Mounted at /content/drive


In [4]:
for i in range(1, 40):
    categories.append(f"{i}")

In [5]:
print(categories)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39']


In [6]:
num_classes = len(categories)

image_w = 100
image_h = 100

pixels = image_w * image_h * 3

x = []
y = []

for idx, category in enumerate(categories):
    label = [0 for i in range(num_classes)]
    label[idx] = 1

    image_dir = data_dir + "/" + category
    files = glob(image_dir + "/*.jpg")

    for i,f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)

        x.append(data)
        y.append(label)

        if i % 1000 == 0:
            print(category, " : ", f)

1  :  /content/drive/My Drive/DeepLearningProject/model/data/1/1_0_2_20161219211823957.jpg.chip.jpg
1  :  /content/drive/My Drive/DeepLearningProject/model/data/1/1_0_2_20161219203016316.jpg.chip.jpg
2  :  /content/drive/My Drive/DeepLearningProject/model/data/2/2_1_2_20161219140736641.jpg.chip.jpg
3  :  /content/drive/My Drive/DeepLearningProject/model/data/3/3_1_2_20161219140825328.jpg.chip.jpg
4  :  /content/drive/My Drive/DeepLearningProject/model/data/4/4_0_2_20161219140938368.jpg.chip.jpg
5  :  /content/drive/My Drive/DeepLearningProject/model/data/5/5_0_2_20161219142128680.jpg.chip.jpg
6  :  /content/drive/My Drive/DeepLearningProject/model/data/6/6_1_2_20161219140554092.jpg.chip.jpg
7  :  /content/drive/My Drive/DeepLearningProject/model/data/7/7_1_2_20161219141129768.jpg.chip.jpg
8  :  /content/drive/My Drive/DeepLearningProject/model/data/8/8_1_2_20161219153017828.jpg.chip.jpg
9  :  /content/drive/My Drive/DeepLearningProject/model/data/9/9_1_2_20161219190524395.jpg.chip.jpg


In [7]:
x = np.array(x)
y = np.array(y)

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y)
xy = (x_train, x_test, y_train, y_test)

#x_train = x_train.astype(float) / 255
#x_test = x_test.astype(float) / 255

In [9]:
input_tensor = Input(shape=(100, 100, 3), dtype='float32', name='input')

#vgg16 모델 불러오기
pre_trained_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(100, 100, 3))
pre_trained_vgg.trainable = False
pre_trained_vgg.summary()

#vgg16 밑에 레이어 추가
add_model = models.Sequential()
add_model.add(pre_trained_vgg)
add_model.add(layers.Flatten())
add_model.add(layers.Dense(4096, kernel_regularizer = regularizers.l1_l2
                                  (l1=0.001,l2=0.001),activation='relu'))
add_model.add(layers.Dropout(0.5))
add_model.add(layers.Dense(2048, kernel_regularizer = regularizers.l1_l2
                                  (l1=0.001,l2=0.001),activation='relu'))
add_model.add(layers.Dropout(0.5))
add_model.add(layers.Dense(1024, kernel_regularizer = regularizers.l1_l2
                                  (l1=0.001,l2=0.001),activation='relu'))
add_model.add(layers.Dropout(0.5))
add_model.add(layers.Dense(39, activation='softmax'))

add_model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

58900480/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                              

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [10]:
add_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 3, 3, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 4096)              18878464  
                                                                 
 dropout (Dropout)           (None, 4096)              0         
                                                                 
 dense_1 (Dense)             (None, 2048)              8390656   
                                                                 
 dropout_1 (Dropout)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 1024)              2

In [11]:
add_model.fit(x_train, y_train, 
                    batch_size=30, 
                    epochs=100, 
                    validation_data=(x_test, y_test))

Epoch 1/100
412/412 [==============================] - 43s 72ms/step - loss: 127.6441 - acc: 0.0840 - val_loss: 4.1411 - val_acc: 0.1328
Epoch 2/100
412/412 [==============================] - 29s 70ms/step - loss: 1.9144 - acc: 0.1145 - val_loss: 1.6330 - val_acc: 0.1328
Epoch 3/100
412/412 [==============================] - 29s 70ms/step - loss: 1.6282 - acc: 0.1286 - val_loss: 1.6120 - val_acc: 0.1328
Epoch 4/100
412/412 [==============================] - 30s 72ms/step - loss: 1.6040 - acc: 0.1325 - val_loss: 1.5910 - val_acc: 0.1328
Epoch 5/100
412/412 [==============================] - 29s 71ms/step - loss: 1.5826 - acc: 0.1336 - val_loss: 1.5792 - val_acc: 0.1328
Epoch 6/100
412/412 [==============================] - 29s 71ms/step - loss: 1.5793 - acc: 0.1336 - val_loss: 1.5793 - val_acc: 0.1328
Epoch 7/100
412/412 [==============================] - 29s 71ms/step - loss: 1.5793 - acc: 0.1336 - val_loss: 1.5792 - val_acc: 0.1328
Epoch 8/100
412/412 [==============================] 

In [12]:
model_json = add_model.to_json()
with open("model.json", "w") as json_file : 
    json_file.write(model_json)